<a href="https://colab.research.google.com/github/KhanhPham2411/DeepLearning_BaNgoc/blob/master/L6/TransferLearning/HumanHorseTransferLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Prepare data

In [1]:
!wget "https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip"
!wget "https://storage.googleapis.com/laurencemoroney-blog.appspot.com/validation-horse-or-human.zip"

--2020-05-18 17:12:14--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.142.128, 2607:f8b0:400e:c09::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.142.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 149574867 (143M) [application/zip]
Saving to: ‘horse-or-human.zip’

horse-or-human.zip  100%[===================>] 142.65M  62.8MB/s    in 2.3s    

2020-05-18 17:12:16 (62.8 MB/s) - ‘horse-or-human.zip’ saved [149574867/149574867]

--2020-05-18 17:12:27--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/validation-horse-or-human.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.142.128, 2607:f8b0:400e:c09::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.142.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11480187 (11M) [application/zip]
Savin

In [0]:
data_folder = "./data"
training_data_folder = data_folder +   '/training'
validation_data_folder = data_folder + '/validation'

In [3]:
import os

def create_folder(folder):
    if not os.path.isdir(folder):
        print('new directry has been created')
        os.system('mkdir {}'.format(folder))

create_folder(data_folder)

new directry has been created


In [0]:
!unzip -q horse-or-human.zip -d $training_data_folder
#bug when upzip, training_data_folder added '\r' behind => os.listdir()

!unzip -q validation-horse-or-human.zip -d $validation_data_folder

In [0]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1/255,
                                     rotation_range = 40,
                                     width_shift_range = 0.2,
                                     height_shift_range = 0.2,
                                     shear_range = 0.2,
                                     zoom_range = 0.2,
                                     horizontal_flip = True)
validation_datagen = ImageDataGenerator(rescale=1/255)

In [6]:
train_generator = train_datagen.flow_from_directory(training_data_folder, 
                                                    batch_size = 20,
                                                    class_mode = 'binary',
                                                    target_size = (150, 150))

validation_generator = validation_datagen.flow_from_directory(validation_data_folder, 
                                                    batch_size = 20,
                                                    class_mode = 'binary',
                                                    target_size = (150, 150))

Found 1027 images belonging to 2 classes.
Found 256 images belonging to 2 classes.


In [7]:
train_generator.class_indices

{'horses': 0, 'humans': 1}

#Prepare transfer model

In [8]:
!wget \
    "https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5"

--2020-05-18 17:12:37--  https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.20.128, 2607:f8b0:400e:c08::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.20.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 87910968 (84M) [application/x-hdf]
Saving to: ‘inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’

inception_v3_weight 100%[===================>]  83.84M  51.9MB/s    in 1.6s    

2020-05-18 17:12:39 (51.9 MB/s) - ‘inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’ saved [87910968/87910968]



In [0]:
from tensorflow.keras.applications.inception_v3 import InceptionV3

In [0]:
local_weights_file = './inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

In [0]:
import os

from tensorflow.keras import layers
from tensorflow.keras import Model

from tensorflow.keras.applications.inception_v3 import InceptionV3

pre_trained_model = InceptionV3(input_shape = (150, 150, 3),
                                include_top = False,
                                weights = None)
pre_trained_model.load_weights(local_weights_file)

In [12]:
pre_trained_model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 150, 150, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 74, 74, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 74, 74, 32)   96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 74, 74, 32)   0           batch_normalization[0][0]        
_______________________________________________________________________________________

In [0]:
for layer in pre_trained_model.layers:
  layer.trainable = False

In [0]:
last_layer = pre_trained_model.get_layer('mixed7')

In [0]:
last_output = last_layer.output

In [16]:
last_output

<tf.Tensor 'mixed7/Identity:0' shape=(None, 7, 7, 768) dtype=float32>

In [0]:
x = layers.Flatten()(last_output)

In [0]:
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(1, activation='sigmoid')(x)

In [0]:
model = Model(pre_trained_model.input, x)

In [20]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 150, 150, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 74, 74, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 74, 74, 32)   96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 74, 74, 32)   0           batch_normalization[0][0]        
______________________________________________________________________________________________

#Train Model

In [0]:
from tensorflow.keras.optimizers import RMSprop

model.compile(optimizer = RMSprop(lr=0.0001),
              loss = 'binary_crossentropy',
              metrics = ['acc'])

In [22]:
history = model.fit_generator(
    train_generator,
    validation_data = validation_generator,
    steps_per_epoch = 100,
    epochs = 20,
    validation_steps = 50,
    verbose =2 
)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/20
100/100 - 25s - loss: 0.0710 - acc: 0.9718 - val_loss: 0.0405 - val_acc: 0.9808
Epoch 2/20
100/100 - 24s - loss: 0.0171 - acc: 0.9919 - val_loss: 0.0750 - val_acc: 0.9766
Epoch 3/20
100/100 - 24s - loss: 0.0245 - acc: 0.9939 - val_loss: 0.0187 - val_acc: 0.9919
Epoch 4/20
100/100 - 24s - loss: 0.0149 - acc: 0.9965 - val_loss: 0.0013 - val_acc: 1.0000
Epoch 5/20
100/100 - 23s - loss: 0.0043 - acc: 0.9995 - val_loss: 0.0489 - val_acc: 0.9878
Epoch 6/20
100/100 - 23s - loss: 0.0115 - acc: 0.9954 - val_loss: 0.1034 - val_acc: 0.9756
Epoch 7/20
100/100 - 23s - loss: 0.0098 - acc: 0.9970 - val_loss: 0.0212 - val_acc: 0.9919
Epoch 8/20
100/100 - 23s - loss: 0.0082 - acc: 0.9965 - val_loss: 0.0181 - val_acc: 0.9878
Epoch 9/20
100/100 - 23s - loss: 0.0118 - acc: 0.9980 - val_loss: 0.0274 - val_acc: 0.9929
Epoch 10/20
100/100 - 22s - loss: 0.0052 - acc: 0.9985 - val_loss: 2.0628e-04 - val_acc: 1.0000
Epoch 11/